# Deep Agents: Building Complex Agents for Long-Horizon Tasks

In this notebook, we'll explore **Deep Agents** - a new approach to building AI agents that can handle complex, multi-step tasks over extended periods. We'll implement all four key elements of Deep Agents while building on our Personal Wellness Assistant use case.

**Learning Objectives:**
- Understand the four key elements of Deep Agents: Planning, Context Management, Subagent Spawning, and Long-term Memory
- Implement each element progressively using the `deepagents` package
- Learn to use Skills for progressive capability disclosure
- Use the `deepagents-cli` for interactive agent sessions

## Table of Contents:

- **Breakout Room #1:** Deep Agent Foundations
  - Task 1: Dependencies & Setup
  - Task 2: Understanding Deep Agents
  - Task 3: Planning with Todo Lists
  - Task 4: Context Management with File Systems
  - Task 5: Basic Deep Agent
  - Question #1 & Question #2
  - Activity #1: Build a Research Agent

- **Breakout Room #2:** Advanced Features & Integration
  - Task 6: Subagent Spawning
  - Task 7: Long-term Memory Integration
  - Task 8: Skills - On-Demand Capabilities
  - Task 9: Using deepagents-cli
  - Task 10: Building a Complete Deep Agent System
  - Question #3 & Question #4
  - Activity #2: Build a Wellness Coach Agent

---
# 🤝 Breakout Room #1
## Deep Agent Foundations

## Task 1: Dependencies & Setup

Before we begin, make sure you have:

1. **API Keys** for:
   - Anthropic (default for Deep Agents) or OpenAI
   - LangSmith (optional, for tracing)
   - Tavily (optional, for web search)

2. **Dependencies installed** via `uv sync`

3. **For the CLI** (Task 9): `uv pip install deepagents-cli`

### Environment Setup

You can either:
- Create a `.env` file with your API keys (recommended):
  ```
  ANTHROPIC_API_KEY=your_key_here
  OPENAI_API_KEY=your_key_here
  LANGCHAIN_API_KEY=your_key_here
  ```
- Or enter them interactively when prompted

In [2]:
# Core imports
import os
import getpass
from uuid import uuid4
from typing import Annotated, TypedDict, Literal

import nest_asyncio
nest_asyncio.apply()  # Required for async operations in Jupyter

# Load environment variables from .env file
from dotenv import load_dotenv
load_dotenv()

def get_api_key(env_var: str, prompt: str) -> str:
    """Get API key from environment or prompt user."""
    value = os.environ.get(env_var, "")
    if not value:
        value = getpass.getpass(prompt)
        if value:
            os.environ[env_var] = value
    return value

In [3]:
# Set Anthropic API Key (default for Deep Agents)
anthropic_key = get_api_key("ANTHROPIC_API_KEY", "Anthropic API Key: ")
if anthropic_key:
    print("Anthropic API key set")
else:
    print("Warning: No Anthropic API key configured")

Anthropic API key set


In [4]:
# Optional: OpenAI for alternative models and subagents
openai_key = get_api_key("OPENAI_API_KEY", "OpenAI API Key (press Enter to skip): ")
if openai_key:
    print("OpenAI API key set")
else:
    print("OpenAI API key not configured (optional)")

OpenAI API key set


In [5]:
# Optional: LangSmith for tracing
langsmith_key = get_api_key("LANGCHAIN_API_KEY", "LangSmith API Key (press Enter to skip): ")

if langsmith_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    os.environ["LANGCHAIN_PROJECT"] = f"AIE9 - Deep Agents - {uuid4().hex[0:8]}"
    print(f"LangSmith tracing enabled. Project: {os.environ['LANGCHAIN_PROJECT']}")
else:
    os.environ["LANGCHAIN_TRACING_V2"] = "false"
    print("LangSmith tracing disabled")

LangSmith tracing enabled. Project: AIE9 - Deep Agents - 3a65c7d9


In [6]:
# Verify deepagents installation
from deepagents import create_deep_agent
print("deepagents package imported successfully!")

# Test with a simple agent
test_agent = create_deep_agent()
result = test_agent.invoke({
    "messages": [{"role": "user", "content": "Say 'Deep Agents ready!' in exactly those words."}]
})
print(result["messages"][-1].content)

deepagents package imported successfully!
Deep Agents ready!


## Task 2: Understanding Deep Agents

**Deep Agents** represent a shift from simple tool-calling loops to sophisticated agents that can handle complex, long-horizon tasks. They address four key challenges:

### The Four Key Elements

| Element | Challenge Addressed | Implementation |
|---------|---------------------|----------------|
| **Planning** | "What should I do?" | Todo lists that persist task state |
| **Context Management** | "What do I know?" | File systems for storing/retrieving info |
| **Subagent Spawning** | "Who can help?" | Task tool for delegating to specialists |
| **Long-term Memory** | "What did I learn?" | LangGraph Store for cross-session memory |

### Deep Agents vs Traditional Agents

```
Traditional Agent Loop:
┌─────────────────────────────────────┐
│  User Query                         │
│       ↓                             │
│  Think → Act → Observe → Repeat     │
│       ↓                             │
│  Response                           │
└─────────────────────────────────────┘
Problems: Context bloat, no delegation,
          loses track of complex tasks

Deep Agent Architecture:
┌─────────────────────────────────────────────────────────┐
│                    Deep Agent                           │
├─────────────────────────────────────────────────────────┤
│  ┌──────────────┐  ┌──────────────┐  ┌──────────────┐   │
│  │   PLANNING   │  │   CONTEXT    │  │   MEMORY     │   │
│  │              │  │  MANAGEMENT  │  │              │   │
│  │ write_todos  │  │              │  │   Store      │   │
│  │ update_todo  │  │  read_file   │  │  namespace   │   │
│  │ list_todos   │  │  write_file  │  │  get/put     │   │
│  │              │  │  edit_file   │  │              │   │
│  └──────────────┘  │  ls          │  └──────────────┘   │
│                    └──────────────┘                     │
│  ┌──────────────────────────────────────────────────┐   │
│  │              SUBAGENT SPAWNING                   │   │
│  │                                                  │   │
│  │  task(prompt, tools, model, system_prompt)       │   │
│  │       ↓              ↓              ↓            │   │
│  │  ┌────────┐    ┌────────┐    ┌────────┐          │   │
│  │  │Research│    │Writing │    │Analysis│          │   │
│  │  │Subagent│    │Subagent│    │Subagent│          │   │
│  │  └────────┘    └────────┘    └────────┘          │   │
│  └──────────────────────────────────────────────────┘   │
└─────────────────────────────────────────────────────────┘
```

### When to Use Deep Agents

| Use Case | Traditional Agent | Deep Agent |
|----------|-------------------|------------|
| Simple Q&A | ✅ | Overkill |
| Single-step tool use | ✅ | Overkill |
| Multi-step research | ⚠️ May lose track | ✅ |
| Complex projects | ❌ Context overflow | ✅ |
| Parallel task execution | ❌ | ✅ |
| Long-running sessions | ❌ | ✅ |

### Key Insight: "Planning is Context Engineering"

Deep Agents treat planning not as a separate phase, but as **context engineering**:
- Todo lists aren't just task trackers—they're **persistent context** about what to do
- File systems aren't just storage—they're **extended memory** beyond the context window
- Subagents aren't just helpers—they're **context isolation** to prevent bloat

## Task 3: Planning with Todo Lists

The first key element of Deep Agents is **Planning**. Instead of trying to hold all task state in the conversation, Deep Agents use structured todo lists.

### Why Todo Lists?

1. **Persistence**: Tasks survive across conversation turns
2. **Visibility**: Both agent and user can see progress
3. **Structure**: Clear tracking of what's done vs pending
4. **Recovery**: Agent can resume from where it left off

### Todo List Tools

| Tool | Purpose |
|------|----------|
| `write_todos` | Create a structured task list |
| `update_todo` | Mark tasks as complete/in-progress |
| `list_todos` | View current task state |

In [7]:
from langchain_core.tools import tool
from typing import List, Optional
import json

# Simple in-memory todo storage for demonstration
# In production, Deep Agents use persistent storage
TODO_STORE = {}

@tool
def write_todos(todos: List[dict]) -> str:
    """Create a list of todos for tracking task progress.
    
    Args:
        todos: List of todo items, each with 'title' and optional 'description'
    
    Returns:
        Confirmation message with todo IDs
    """
    created = []
    for i, todo in enumerate(todos):
        todo_id = f"todo_{len(TODO_STORE) + i + 1}"
        TODO_STORE[todo_id] = {
            "id": todo_id,
            "title": todo.get("title", "Untitled"),
            "description": todo.get("description", ""),
            "status": "pending"
        }
        created.append(todo_id)
    return f"Created {len(created)} todos: {', '.join(created)}"

@tool
def update_todo(todo_id: str, status: Literal["pending", "in_progress", "completed"]) -> str:
    """Update the status of a todo item.
    
    Args:
        todo_id: The ID of the todo to update
        status: New status (pending, in_progress, completed)
    
    Returns:
        Confirmation message
    """
    if todo_id not in TODO_STORE:
        return f"Todo {todo_id} not found"
    TODO_STORE[todo_id]["status"] = status
    return f"Updated {todo_id} to {status}"

@tool
def list_todos() -> str:
    """List all todos with their current status.
    
    Returns:
        Formatted list of all todos
    """
    if not TODO_STORE:
        return "No todos found"
    
    result = []
    for todo_id, todo in TODO_STORE.items():
        status_emoji = {"pending": "⬜", "in_progress": "🔄", "completed": "✅"}
        emoji = status_emoji.get(todo["status"], "❓")
        result.append(f"{emoji} [{todo_id}] {todo['title']} ({todo['status']})")
    return "\n".join(result)

print("Todo tools defined!")

Todo tools defined!


In [8]:
# Test the todo tools
TODO_STORE.clear()  # Reset for demo

# Create some wellness todos
result = write_todos.invoke({
    "todos": [
        {"title": "Assess current sleep patterns", "description": "Review user's sleep schedule and quality"},
        {"title": "Research sleep improvement strategies", "description": "Find evidence-based techniques"},
        {"title": "Create personalized sleep plan", "description": "Combine findings into actionable steps"},
    ]
})
print(result)
print("\nCurrent todos:")
print(list_todos.invoke({}))

Created 3 todos: todo_1, todo_3, todo_5

Current todos:
⬜ [todo_1] Assess current sleep patterns (pending)
⬜ [todo_3] Research sleep improvement strategies (pending)
⬜ [todo_5] Create personalized sleep plan (pending)


In [9]:
# Simulate progress
update_todo.invoke({"todo_id": "todo_1", "status": "completed"})
update_todo.invoke({"todo_id": "todo_3", "status": "in_progress"})

print("After updates:")
print(list_todos.invoke({}))

After updates:
✅ [todo_1] Assess current sleep patterns (completed)
🔄 [todo_3] Research sleep improvement strategies (in_progress)
⬜ [todo_5] Create personalized sleep plan (pending)


## Task 4: Context Management with File Systems

The second key element is **Context Management**. Deep Agents use file systems to:

1. **Offload large content** - Store research, documents, and results to disk
2. **Persist across sessions** - Files survive beyond conversation context
3. **Share between subagents** - Subagents can read/write shared files
4. **Prevent context overflow** - Large tool results automatically saved to disk

### Automatic Context Management

Deep Agents automatically handle context limits:
- **Large result offloading**: Tool results >20k tokens → saved to disk
- **Proactive offloading**: At 85% context capacity → agent saves state to disk
- **Summarization**: Long conversations get summarized while preserving intent

### File System Tools

| Tool | Purpose |
|------|----------|
| `ls` | List directory contents |
| `read_file` | Read file contents |
| `write_file` | Create/overwrite files |
| `edit_file` | Make targeted edits |

In [10]:
import os
from pathlib import Path

# Create a workspace directory for our agent
WORKSPACE = Path("workspace")
WORKSPACE.mkdir(exist_ok=True)

@tool
def ls(path: str = ".") -> str:
    """List contents of a directory.
    
    Args:
        path: Directory path to list (default: current directory)
    
    Returns:
        List of files and directories
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"Directory not found: {path}"
    
    items = []
    for item in sorted(target.iterdir()):
        prefix = "[DIR]" if item.is_dir() else "[FILE]"
        size = f" ({item.stat().st_size} bytes)" if item.is_file() else ""
        items.append(f"{prefix} {item.name}{size}")
    
    return "\n".join(items) if items else "(empty directory)"

@tool
def read_file(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()

@tool
def write_file(path: str, content: str) -> str:
    """Write content to a file (creates or overwrites).
    
    Args:
        path: Path to the file to write
        content: Content to write to the file
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    target.parent.mkdir(parents=True, exist_ok=True)
    target.write_text(content)
    return f"Wrote {len(content)} characters to {path}"

@tool
def edit_file(path: str, old_text: str, new_text: str) -> str:
    """Edit a file by replacing text.
    
    Args:
        path: Path to the file to edit
        old_text: Text to find and replace
        new_text: Replacement text
    
    Returns:
        Confirmation message
    """
    target = WORKSPACE / path
    if not target.exists():
        return f"File not found: {path}"
    
    content = target.read_text()
    if old_text not in content:
        return f"Text not found in {path}"
    
    new_content = content.replace(old_text, new_text, 1)
    target.write_text(new_content)
    return f"Updated {path}"

print("File system tools defined!")
print(f"Workspace: {WORKSPACE.absolute()}")

File system tools defined!
Workspace: /Users/swarabankhele/aimakers/github/bootcamp/AIE9/07_Deep_Agents/workspace


In [11]:
# Test the file system tools
print("Current workspace contents:")
print(ls.invoke({"path": "."}))

Current workspace contents:
[FILE] Alex_2_Week_Exercise_Program.txt (4541 bytes)
[FILE] alex_exercise_program.txt (4046 bytes)
[FILE] alex_integrated_2week_wellness_program.txt (11071 bytes)
[FILE] alex_nutrition_plan.txt (6181 bytes)
[FILE] alex_nutrition_plan_week_1.txt (5451 bytes)
[FILE] alex_nutrition_plan_week_2.txt (3222 bytes)
[FILE] alex_stress_management_sleep_program.txt (8373 bytes)
[FILE] comprehensive_morning_routine_guide.md (15151 bytes)
[FILE] comprehensive_stress_management_guide.txt (13320 bytes)
[FILE] comprehensive_stress_management_research_guide.md (20959 bytes)
[FILE] final_report.md (28868 bytes)
[FILE] morning_routine_guide.md (28150 bytes)
[FILE] my_sleep_improvement_plan.md (4594 bytes)
[FILE] personalized_sleep_improvement_plan.md (6628 bytes)
[DIR] research
[FILE] sleep_improvement_quick_guide.md (3858 bytes)
[FILE] sleep_improvement_research.md (18713 bytes)
[FILE] stress_management_research.md (29523 bytes)
[FILE] stress_management_sleep_optimization_pro

In [12]:
# Create a research notes file
notes = """# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations
"""

result = write_file.invoke({"path": "research/sleep_notes.md", "content": notes})
print(result)

# Verify it was created
print("\nResearch directory:")
print(ls.invoke({"path": "research"}))

Wrote 242 characters to research/sleep_notes.md

Research directory:
[FILE] sleep_notes.md (242 bytes)


In [13]:
# Read and edit the file
print("File contents:")
print(read_file.invoke({"path": "research/sleep_notes.md"}))

File contents:
# Sleep Research Notes

## Key Findings
- Adults need 7-9 hours of sleep
- Consistent sleep schedule is important
- Blue light affects melatonin production

## TODO
- [ ] Review individual user needs
- [ ] Create personalized recommendations



## Task 5: Basic Deep Agent

Now let's create a basic Deep Agent using the `deepagents` package. This combines:
- Planning (todo lists)
- Context management (file system)
- A capable LLM backbone

### Configuring the FilesystemBackend

Deep Agents come with **built-in file tools** (`ls`, `read_file`, `write_file`, `edit_file`). To control where files are stored, we configure a `FilesystemBackend`:

```python
from deepagents.backends import FilesystemBackend

backend = FilesystemBackend(
    root_dir="/path/to/workspace",
    virtual_mode=True  # REQUIRED to actually sandbox files!
)
```

**Critical: `virtual_mode=True`**
- Without `virtual_mode=True`, agents can still write anywhere on the filesystem!
- The `root_dir` alone does NOT restrict file access
- `virtual_mode=True` blocks paths with `..`, `~`, and absolute paths outside root

In [14]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Configure the filesystem backend to use our workspace directory
# IMPORTANT: virtual_mode=True is required to actually restrict paths to root_dir
# Without it, agents can still write anywhere on the filesystem!
workspace_path = Path("workspace").absolute()
filesystem_backend = FilesystemBackend(
    root_dir=str(workspace_path),
    virtual_mode=True  # This is required to sandbox file operations!
)

# Combine our custom tools (for todo tracking)
# Note: Deep Agents has built-in file tools (ls, read_file, write_file, edit_file)
# that will use the configured FilesystemBackend
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
]

# Create a basic Deep Agent
wellness_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=filesystem_backend,  # Configure where files are stored
    system_prompt="""You are a Personal Wellness Assistant that helps users improve their health.

When given a complex task:
1. First, create a todo list to track your progress
2. Work through each task, updating status as you go
3. Save important findings to files for reference
4. Provide a clear summary when complete

Be thorough but concise. Always explain your reasoning."""
)

print(f"Basic Deep Agent created!")
print(f"File operations sandboxed to: {workspace_path}")

Basic Deep Agent created!
File operations sandboxed to: /Users/swarabankhele/aimakers/github/bootcamp/AIE9/07_Deep_Agents/workspace


In [15]:
# Reset todo store for fresh demo
TODO_STORE.clear()

# Test with a multi-step wellness task
result = wellness_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """I want to improve my sleep quality. I currently:
- Go to bed at inconsistent times (10pm-1am)
- Use my phone in bed
- Often feel tired in the morning

Please create a personalized sleep improvement plan for me and save it to a file."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Perfect! I've created your personalized sleep improvement plan and saved it to `/sleep_improvement_plan_2024.md`. 

## Summary

Your plan addresses your three key issues with targeted, evidence-based solutions:

**🕙 Inconsistent Bedtime (10pm-1am)** → **Fixed 10:30 PM bedtime**
- Gradual 30-minute adjustments every 2-3 days
- Wind-down alarm at 9:30 PM
- Consistent schedule even on weekends

**📱 Phone Use in Bed** → **Complete evening digital detox**
- Phone goes to another room at 10 PM (Week 1), then 9 PM (Week 3)
- Alternative wind-down activities provided
- Emergency contact plan included

**😴 Morning Fatigue** → **Circadian rhythm reset**
- 7 AM wake time daily (no weekends off!)
- Immediate bright light exposure upon waking
- No snooze button allowed

## Key Success Factors:
1. **Start gradually** - don't try to change everything at once
2. **Be consistent** - even small improvements compound over time  
3. **Track progress** - daily sleep quality ratings help you

In [16]:
# Check what the agent created
print("Todo list after task:")
print(list_todos.invoke({}))

print("\n" + "="*50)
print("\nWorkspace contents:")
# List files in the workspace directory
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    else:
        print(f"  [DIR] {f.name}/")

Todo list after task:
⬜ [todo_1] Analyze current sleep issues (pending)
✅ [todo_3] Research sleep hygiene best practices (completed)
✅ [todo_5] Create personalized sleep improvement plan (completed)
✅ [todo_7] Save plan to file (completed)


Workspace contents:
  [FILE] Alex_2_Week_Exercise_Program.txt (4541 bytes)
  [FILE] alex_exercise_program.txt (4046 bytes)
  [FILE] alex_integrated_2week_wellness_program.txt (11071 bytes)
  [FILE] alex_nutrition_plan.txt (6181 bytes)
  [FILE] alex_nutrition_plan_week_1.txt (5451 bytes)
  [FILE] alex_nutrition_plan_week_2.txt (3222 bytes)
  [FILE] alex_stress_management_sleep_program.txt (8373 bytes)
  [FILE] comprehensive_morning_routine_guide.md (15151 bytes)
  [FILE] comprehensive_stress_management_guide.txt (13320 bytes)
  [FILE] comprehensive_stress_management_research_guide.md (20959 bytes)
  [FILE] final_report.md (28868 bytes)
  [FILE] morning_routine_guide.md (28150 bytes)
  [FILE] my_sleep_improvement_plan.md (4594 bytes)
  [FILE] persona

---
## ❓ Question #1:

What are the **trade-offs** of using todo lists for planning? Consider:
- When might explicit planning overhead slow things down?
- How granular should todo items be?
- What happens if the agent creates todos but never completes them?

##### Answer:
Here is a breakdown of the trade-offs involved.
1. The Planning Overhead vs. Execution Speed
Explicit planning is essentially Meta-Reasoning. While it helps with complex, multi-step goals, it comes at a cost.
    a. Latency: Every time an agent stops to "update the list," it consumes tokens and time. In simple tasks (e.g., "Find the capital of France"), explicit planning is a waste of resources.
    b. The "Analysis Paralysis" Loop: If an agent is programmed to re-evaluate its plan after every single action, it can get stuck in a loop of over-planning—constantly refining the list rather than actually interacting with the environment.
    c. When to Skip It: Reactive tasks or high-speed environments (like real-time gaming or live chat support) often benefit more from Chain-of-Thought (CoT) reasoning rather than a formal, persistent To-Do list.

2. Granularity: The Goldilocks Problem
The "size" of a task determines whether an agent succeeds or wanders off-track.
    a. Coarse grain tasks helps with low token usage and high speed but the agent will likely hallucinate or skip crucial sub-steps 
    b. Fine grained tasks helps with high reliability and precision but massive overhead, the agent may lose the "big picture" (context drift) in the minutiae.

Task items should ideally represent one atomic API call or tool use. If a To-Do item requires more than one distinct tool (e.g., "Search for X and summarize it"), it's usually too broad.

3. The "Zombie Task" Phenomenon
What happens when an agent creates tasks but never checks them off? This is a common failure mode in autonomous systems.
    a. Context Saturation: As the To-Do list grows with uncompleted tasks, the agent’s "context window" fills up with old information. This leads to diluted attention, where the agent forgets the primary goal because it's staring at 20 unfinished sub-tasks.
    b. Recursive Loops: If an agent fails a task and simply re-adds it to the list without changing the strategy, it creates an infinite loop of failure.
    c. The Solution (Pruning): Modern agent architectures (like AutoGPT or BabyAGI descendants) require a Re-prioritization Step. The agent must be capable of "Garbage Collection"—deleting tasks that are no longer relevant or realizing a path is a dead end

## ❓ Question #2:

How would you design a **context management strategy** for a wellness agent that:
- Needs to reference a large health document (16KB)
- Tracks user metrics over time
- Must remember user conditions (allergies, medications) for safety

What goes in files vs. in the prompt? What should never be offloaded?

##### Answer:
Designing a context management strategy for a wellness agent requires a balance between reliability (safety first!), scalability (handling that 16KB doc), and efficiency.

1. System Prompt (The "Never Forget" Zone)
The system prompt should act as the agent’s "Operating Manual."
    a. Static Safety Profile: Explicitly list allergies and current medications.
    b. Operational Constraints: Instructions like "If a user asks for a workout but has a knee injury (from their file), suggest low-impact only."
    c. Persona & Guardrails: The "voice" and the hard limit that it is an AI, not a doctor.

2. In Files/Knowledge Base (The "Encyclopedia")
The 16KB health document is a prime candidate for Retrieval-Augmented Generation (RAG) or a File Attachment.
    a. Segmented Retrieval: Instead of feeding the whole 16KB every time, the system should search the file for keywords (e.g., "Post-op recovery protocols") only when the user asks about them.
    b. Detailed References: Nuanced medical guidelines or complex program details.

3. In the Conversation History (The "Recent Context")
    a. Short-term memory: The last 5–10 exchanges to maintain the "flow."
    b. Metric Delta: If a user logs a metric, the agent should acknowledge the change relative to the previous entry (e.g., "You slept 2 hours more than yesterday").

Following should never be offloaded:
    a. Life-Threatening Allergies: If the user is allergic to Peanuts, that must be in the active context window at all times. You cannot risk the RAG system failing to "retrieve" the allergy note while the agent suggests a snack.
    b. Current Emergency Protocols: If the user is in a specific high-risk recovery phase, that status stays in the prompt.
    c. Active Intent: What the user is trying to achieve right now in this session.

---
## 🏗️ Activity #1: Build a Research Agent
Build a Deep Agent that can research a wellness topic and produce a structured report.

### Requirements:
1. Create todos for the research process
2. Read from the HealthWellnessGuide.txt in the data folder
3. Save findings to a structured markdown file
4. Update todo status as tasks complete


### Test prompt:
"Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."

In [ ]:
### YOUR CODE HERE ###

# Step 1: Create a research agent with appropriate tools
# Hint: You'll need file tools to read the wellness guide


# Step 2: Add a tool to read from the data folder
# Hint: Use Path("data/HealthWellnessGuide.txt")
from deepagents.backends import CompositeBackend, FilesystemBackend

data_path = Path("data").absolute() 

composite = CompositeBackend(
    default=FilesystemBackend(root_dir=str(workspace_path), virtual_mode=True),
    routes={
        "/data/": FilesystemBackend(root_dir=str(data_path), virtual_mode=True),
    },
)

@tool
def read_file_from_data(path: str) -> str:
    """Read contents of a file.
    
    Args:
        path: Path to the file to read
    
    Returns:
        File contents
    """
    target = data_path / path
    if not target.exists():
        return f"File not found: {path}"
    return target.read_text()


# Step 3: Create the agent with a research-focused system prompt
custom_tools = [
    write_todos,
    update_todo,
    list_todos,
    read_file_from_data,
]

research_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=custom_tools,
    backend=composite,  # Configure where files are stored
    system_prompt="""You are a Wellness Research Assistant that helps users research wellness topics.

When given a complex task:
1. Create todos for: read guide, extract stress management content, draft report, save report.
2. Use your custom tool (or adjusted backend) to read data/HealthWellnessGuide.txt.
3. Extract and synthesize stress management strategies.
4. Update todos as steps complete.
5. Write the structured report with write_file in final_report.md.

Be thorough but concise. Always explain your reasoning."""
)

TODO_STORE.clear()

# Step 4: Test with the stress management research task
result = research_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Research stress management techniques and create a comprehensive guide with at least 5 evidence-based strategies."""
    }]
})

print("Agent response:")
print(result["messages"][-1].content)


Agent response:
Perfect! I've successfully completed your comprehensive stress management guide. Here's what I accomplished:

## Summary of the Comprehensive Stress Management Guide

I've created a thorough, evidence-based guide that includes **10 scientifically-backed stress management strategies**, each with:

### The 10 Evidence-Based Strategies:

1. **Deep Breathing & Respiratory Techniques** - Including 4-7-8 breathing and Box Breathing with proven cortisol reduction
2. **Progressive Muscle Relaxation (PMR)** - Systematic tension/release technique with 35% muscle tension reduction
3. **Mindfulness Meditation** - Various forms including body scan and loving-kindness with 60% anxiety reduction
4. **Physical Exercise for Stress Relief** - HIIT, yoga, and cardio with up to 40% stress hormone reduction
5. **Cognitive Behavioral Techniques** - CBT methods, cognitive restructuring, and STOP technique with 65% symptom reduction
6. **Expressive Writing Therapy** - Pennebaker protocol with 

---
# 🤝 Breakout Room #2
## Advanced Features & Integration

## Task 6: Subagent Spawning

The third key element is **Subagent Spawning**. This allows a Deep Agent to delegate tasks to specialized subagents.

### Why Subagents?

1. **Context Isolation**: Each subagent has its own context window, preventing bloat
2. **Specialization**: Different subagents can have different tools/prompts
3. **Parallelism**: Multiple subagents can work simultaneously
4. **Cost Optimization**: Use cheaper models for simpler subtasks

### How Subagents Work

```
Main Agent
    ├── task("Research sleep science", model="gpt-4o-mini")
    │       └── Returns: Summary of findings
    │
    ├── task("Analyze user's sleep data", tools=[analyze_tool])
    │       └── Returns: Analysis results
    │
    └── task("Write recommendations", system_prompt="Be concise")
            └── Returns: Final recommendations
```

Key benefit: The main agent only receives **summaries**, not all the intermediate context!

In [ ]:
from deepagents import create_deep_agent
from deepagents.backends import FilesystemBackend
from langchain.chat_models import init_chat_model

# Define specialized subagent configurations
# Note: Subagents inherit the backend from the parent agent
research_subagent = {
    "name": "research-agent",
    "description": "Use this agent to research wellness topics in depth. It can read documents and synthesize information.",
    "system_prompt": """You are a wellness research specialist. Your job is to:
1. Find relevant information in provided documents
2. Synthesize findings into clear summaries
3. Cite sources when possible

Be thorough but concise. Focus on evidence-based information.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",  # Cheaper model for research
}

writing_subagent = {
    "name": "writing-agent",
    "description": "Use this agent to create well-structured documents, plans, and guides.",
    "system_prompt": """You are a wellness content writer. Your job is to:
1. Take research findings and turn them into clear, actionable content
2. Structure information for easy understanding
3. Use formatting (headers, bullets, etc.) effectively

Write in a supportive, encouraging tone.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "anthropic:claude-sonnet-4-20250514",
}

print("Subagent configurations defined!")

Subagent configurations defined!


In [ ]:
# Create a coordinator agent that can spawn subagents
coordinator_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[write_todos, update_todo, list_todos],
    backend=filesystem_backend,  # Use the same backend - subagents inherit it
    subagents=[research_subagent, writing_subagent],
    system_prompt="""You are a Wellness Project Coordinator. Your role is to:
1. Break down complex wellness requests into subtasks
2. Delegate research to the research-agent
3. Delegate content creation to the writing-agent
4. Coordinate the overall workflow using todos

Use subagents for specialized work rather than doing everything yourself.
This keeps the work organized and the results high-quality."""
)

print("Coordinator agent created with subagent capabilities!")

Coordinator agent created with subagent capabilities!


In [20]:
# Reset for demo
TODO_STORE.clear()

# Test the coordinator with a complex task
result = coordinator_agent.invoke({
    "messages": [{
        "role": "user",
        "content": """Create a comprehensive morning routine guide for better energy.
        
The guide should:
1. Research the science behind morning routines
2. Include practical steps for exercise, nutrition, and mindset
3. Be saved as a well-formatted markdown file"""
    }]
})

print("Coordinator response:")
print(result["messages"][-1].content)

Coordinator response:
## 🎉 Project Complete! 

I've successfully created a comprehensive morning routine guide for better energy that meets all your requirements. Here's what was accomplished:

### ✅ **What We Delivered:**

**📋 Project Management:**
- Used coordinated workflow with specialized agents
- Tracked progress with todos throughout the process
- Completed all planned subtasks systematically

**🔬 Science-Backed Research:**
- Research-agent conducted thorough investigation of circadian rhythms, exercise benefits, nutrition science, and mindset practices
- Incorporated findings on cortisol optimization, endorphin release, and neurochemistry
- Evidence-based timing recommendations throughout

**📝 Comprehensive Guide Creation:**
- Writing-agent crafted a detailed, well-structured guide
- **9 main sections** covering all requested areas
- **Multiple implementation options** for different lifestyles and time constraints

**📁 Final Deliverable:**
- **Saved as:** `/complete_morning_rou

In [21]:
# Check the results
print("Final todo status:")
print(list_todos.invoke({}))

print("\nGenerated files in workspace:")
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

Final todo status:
✅ [todo_1] Research the science behind morning routines and energy (completed)
✅ [todo_3] Create comprehensive morning routine guide (completed)
✅ [todo_5] Review and finalize the guide (completed)

Generated files in workspace:
  [FILE] Alex_2_Week_Exercise_Program.txt (4541 bytes)
  [FILE] alex_exercise_program.txt (4046 bytes)
  [FILE] alex_integrated_2week_wellness_program.txt (11071 bytes)
  [FILE] alex_nutrition_plan.txt (6181 bytes)
  [FILE] alex_nutrition_plan_week_1.txt (5451 bytes)
  [FILE] alex_nutrition_plan_week_2.txt (3222 bytes)
  [FILE] alex_stress_management_sleep_program.txt (8373 bytes)
  [FILE] complete_morning_routine_energy_guide_2024.md (15151 bytes)
  [FILE] comprehensive_morning_routine_guide.md (15151 bytes)
  [FILE] comprehensive_stress_management_guide.md (13050 bytes)
  [FILE] comprehensive_stress_management_guide.txt (13320 bytes)
  [FILE] comprehensive_stress_management_research_guide.md (20959 bytes)
  [FILE] evidence_based_stress_mana

## Task 7: Long-term Memory Integration

The fourth key element is **Long-term Memory**. Deep Agents integrate with LangGraph's Store for persistent memory across sessions.

### Memory Types in Deep Agents

| Type | Scope | Use Case |
|------|-------|----------|
| **Thread Memory** | Single conversation | Current session context |
| **User Memory** | Across threads, per user | User preferences, history |
| **Shared Memory** | Across all users | Common knowledge, learned patterns |

### Integration with LangGraph Store

Deep Agents can use the same `InMemoryStore` (or `PostgresStore`) we learned in Session 6:

In [22]:
from langgraph.store.memory import InMemoryStore

# Create a memory store
memory_store = InMemoryStore()

# Store user profile
user_id = "user_alex"
profile_namespace = (user_id, "profile")

memory_store.put(profile_namespace, "name", {"value": "Alex"})
memory_store.put(profile_namespace, "goals", {
    "primary": "improve energy levels",
    "secondary": "better sleep"
})
memory_store.put(profile_namespace, "conditions", {
    "dietary": ["vegetarian"],
    "medical": ["mild anxiety"]
})
memory_store.put(profile_namespace, "preferences", {
    "exercise_time": "morning",
    "communication_style": "detailed"
})

print(f"Stored profile for {user_id}")

# Retrieve and display
for item in memory_store.search(profile_namespace):
    print(f"  {item.key}: {item.value}")

Stored profile for user_alex
  name: {'value': 'Alex'}
  goals: {'primary': 'improve energy levels', 'secondary': 'better sleep'}
  conditions: {'dietary': ['vegetarian'], 'medical': ['mild anxiety']}
  preferences: {'exercise_time': 'morning', 'communication_style': 'detailed'}


In [23]:
# Create memory-aware tools
from langgraph.store.base import BaseStore

@tool
def get_user_profile(user_id: str) -> str:
    """Retrieve a user's wellness profile from long-term memory.
    
    Args:
        user_id: The user's unique identifier
    
    Returns:
        User profile as formatted text
    """
    namespace = (user_id, "profile")
    items = list(memory_store.search(namespace))
    
    if not items:
        return f"No profile found for {user_id}"
    
    result = [f"Profile for {user_id}:"]
    for item in items:
        result.append(f"  {item.key}: {item.value}")
    return "\n".join(result)

@tool
def save_user_preference(user_id: str, key: str, value: str) -> str:
    """Save a user preference to long-term memory.
    
    Args:
        user_id: The user's unique identifier
        key: The preference key
        value: The preference value
    
    Returns:
        Confirmation message
    """
    namespace = (user_id, "preferences")
    memory_store.put(namespace, key, {"value": value})
    return f"Saved preference '{key}' for {user_id}"

print("Memory tools defined!")

Memory tools defined!


In [24]:
# Create a memory-enhanced agent
memory_tools = [
    get_user_profile,
    save_user_preference,
    write_todos,
    update_todo,
    list_todos,
]

memory_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=memory_tools,
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a Personal Wellness Assistant with long-term memory.

At the start of each conversation:
1. Check the user's profile to understand their goals and conditions
2. Personalize all advice based on their profile
3. Save any new preferences they mention

Always reference stored information to show you remember the user."""
)

print("Memory-enhanced agent created!")

Memory-enhanced agent created!


In [25]:
# Test the memory agent
TODO_STORE.clear()

result = memory_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "Hi! My user_id is user_alex. What exercise routine would you recommend for me?"
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
Great to see you again, Alex! Based on your profile, I can see your primary goal is to improve energy levels with better sleep as a secondary goal. I also note you prefer morning exercise and have mild anxiety. Here's a personalized exercise routine that should work well for you:

## Morning Energy-Boosting Exercise Routine

**Duration:** 20-30 minutes (perfect for mornings!)

### Week Structure:
- **Monday, Wednesday, Friday:** Cardio + Light Strength
- **Tuesday, Thursday:** Yoga/Stretching + Core
- **Saturday:** Active recovery (gentle walk or light activity)
- **Sunday:** Rest

### Daily Breakdown:

**Cardio + Strength Days (MWF):**
1. **Warm-up** (5 minutes): Light marching in place, arm circles, gentle stretches
2. **Cardio** (10 minutes): Choose one:
   - Brisk walk outside (great for morning sunlight exposure)
   - Dance to upbeat music
   - Jumping jacks/modified burpees
3. **Strength** (10 minutes): Bodyweight exercises
   - Push-ups (wall or knee variations i

## Task 8: Skills - On-Demand Capabilities

**Skills** are a powerful feature for progressive capability disclosure. Instead of loading all tools upfront, agents can load specialized capabilities on demand.

### Why Skills?

1. **Context Efficiency**: Don't waste context on unused tool descriptions
2. **Specialization**: Skills can include detailed instructions for specific tasks
3. **Modularity**: Easy to add/remove capabilities
4. **Discoverability**: Agent can browse available skills

### SKILL.md Format

Skills are defined in markdown files with YAML frontmatter:

```markdown
---
name: skill-name
description: What this skill does
version: 1.0.0
tools:
  - tool1
  - tool2
---

# Skill Instructions

Detailed steps for how to use this skill...
```

In [26]:
# Let's look at the skills we created
skills_dir = Path("skills")

print("Available skills:")
for skill_dir in skills_dir.iterdir():
    if skill_dir.is_dir():
        skill_file = skill_dir / "SKILL.md"
        if skill_file.exists():
            content = skill_file.read_text()
            # Extract name and description from frontmatter
            lines = content.split("\n")
            name = ""
            desc = ""
            for line in lines:
                if line.startswith("name:"):
                    name = line.split(":", 1)[1].strip()
                if line.startswith("description:"):
                    desc = line.split(":", 1)[1].strip()
            print(f"  - {name}: {desc}")

Available skills:
  - meal-planning: Create personalized meal plans based on dietary needs and preferences
  - wellness-assessment: Assess user wellness goals and create personalized recommendations


In [27]:
# Read the wellness-assessment skill
skill_content = Path("skills/wellness-assessment/SKILL.md").read_text()
print(skill_content)

---
name: wellness-assessment
description: Assess user wellness goals and create personalized recommendations
version: 1.0.0
tools:
  - read_file
  - write_file
---

# Wellness Assessment Skill

You are conducting a comprehensive wellness assessment. Follow these steps:

## Step 1: Gather Information
Ask the user about:
- Current health goals (weight, fitness, stress, sleep)
- Any medical conditions or limitations
- Current exercise routine (or lack thereof)
- Dietary preferences and restrictions
- Sleep patterns and quality
- Stress levels and sources

## Step 2: Analyze Responses
Review the user's answers and identify:
- Primary wellness priority
- Secondary goals
- Potential barriers to success
- Existing healthy habits to build on

## Step 3: Create Assessment Report
Write a wellness assessment report to `workspace/wellness_assessment.md` containing:
- Summary of current wellness state
- Identified strengths
- Areas for improvement
- Recommended focus areas (prioritized)
- Suggeste

In [28]:
# Create a skill-aware tool
@tool
def load_skill(skill_name: str) -> str:
    """Load a skill's instructions for a specialized task.
    
    Available skills:
    - wellness-assessment: Assess user wellness and create recommendations
    - meal-planning: Create personalized meal plans
    
    Args:
        skill_name: Name of the skill to load
    
    Returns:
        Skill instructions
    """
    skill_path = Path(f"skills/{skill_name}/SKILL.md")
    if not skill_path.exists():
        available = [d.name for d in Path("skills").iterdir() if d.is_dir()]
        return f"Skill '{skill_name}' not found. Available: {', '.join(available)}"
    
    return skill_path.read_text()

print("Skill loader defined!")

Skill loader defined!


In [29]:
# Create an agent that can load and use skills
skill_agent = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        load_skill,
        write_todos,
        update_todo,
        list_todos,
    ],
    backend=filesystem_backend,  # Use workspace for file operations
    system_prompt="""You are a wellness assistant with access to specialized skills.

When a user asks for something that matches a skill:
1. Load the appropriate skill using load_skill()
2. Follow the skill's instructions carefully
3. Save outputs as specified in the skill

Available skills:
- wellness-assessment: For comprehensive wellness evaluations
- meal-planning: For creating personalized meal plans

If no skill matches, use your general wellness knowledge."""
)

print("Skill-aware agent created!")

Skill-aware agent created!


In [30]:
# Test with a skill-appropriate request
TODO_STORE.clear()

result = skill_agent.invoke({
    "messages": [{
        "role": "user",
        "content": "I'd like a wellness assessment. I'm a 35-year-old office worker who sits most of the day, has trouble sleeping, and wants to lose 15 pounds. I'm vegetarian and have no major health conditions."
    }]
})

print("Agent response:")
print(result["messages"][-1].content)

Agent response:
## Your Wellness Assessment Results

Based on your profile, I've identified that **movement integration** and **sleep optimization** are your top priorities, as they directly impact each other and your weight loss goal.

### 🎯 **Immediate Action Items** (Start Today)
1. **Set hourly movement reminders** - Stand and walk for 2 minutes every hour during work
2. **Create a wind-down routine** - No screens 1 hour before bed, try reading or gentle stretching
3. **Plan tomorrow's meals** - Prep a high-protein vegetarian breakfast and lunch to avoid impulsive eating

### 📅 **Short-term Goals** (1-2 weeks)
1. **Establish a 10-minute morning routine** - Light stretching or yoga to energize your day
2. **Track your sleep** - Note bedtime, wake time, and sleep quality to identify patterns
3. **Add walking meetings** - Suggest phone calls while walking when possible

### 🏆 **Long-term Goals** (1-3 months)
1. **Build a 30-minute exercise routine** - 3-4 times per week, mixing cardio

## Task 9: Using deepagents-cli

The `deepagents-cli` provides an interactive terminal interface for working with Deep Agents.

### Installation

```bash
uv pip install deepagents-cli
# or
pip install deepagents-cli
```

### Key Features

| Feature | Description |
|---------|-------------|
| **Interactive Sessions** | Chat with your agent in the terminal |
| **Conversation Resume** | Pick up where you left off |
| **Human-in-the-Loop** | Approve or reject agent actions |
| **File System Access** | Agent can read/write to your filesystem |
| **Remote Sandboxing** | Run in isolated Docker containers |

### Basic Usage

```bash
# Start an interactive session
deepagents

# Resume a previous conversation
deepagents --resume

# Use a specific model
deepagents --model openai:gpt-4o

# Enable human-in-the-loop approval
deepagents --approval-mode full
```

### Example Session

```
$ deepagents

Welcome to Deep Agents CLI!

You: Create a 7-day meal plan for a vegetarian athlete

Agent: I'll create a comprehensive meal plan for you. Let me:
1. Research vegetarian athlete nutrition needs
2. Design balanced daily menus
3. Save the plan to a file

[Agent uses tools...]

Agent: I've created your meal plan! You can find it at:
workspace/vegetarian_athlete_meal_plan.md

You: /exit
```

In [31]:
# Check if CLI is installed
import subprocess

try:
    result = subprocess.run(["deepagents", "--version"], capture_output=True, text=True)
    print(f"deepagents-cli version: {result.stdout.strip()}")
except FileNotFoundError:
    print("deepagents-cli not installed. Install with:")
    print("  uv pip install deepagents-cli")
    print("  # or")
    print("  pip install deepagents-cli")

deepagents-cli not installed. Install with:
  uv pip install deepagents-cli
  # or
  pip install deepagents-cli


### Try It Yourself!

After installing the CLI, try these commands in your terminal:

```bash
# Basic interactive session
deepagents

# With a specific working directory
deepagents --workdir ./workspace

# See all options
deepagents --help
```

Sample prompts to try:
1. "Create a weekly workout plan and save it to a file"
2. "Research the health benefits of meditation and summarize in a report"
3. "Analyze my current diet and suggest improvements" (then provide details)

## Task 10: Building a Complete Deep Agent System

Now let's bring together all four elements to build a comprehensive "Wellness Coach" system:

1. **Planning**: Track multi-week wellness programs
2. **Context Management**: Store session notes and progress
3. **Subagent Spawning**: Delegate to specialists (exercise, nutrition, mindfulness)
4. **Long-term Memory**: Remember user preferences and history

In [32]:
# Define specialized wellness subagents
# Subagents inherit the backend from the parent, so they use the same workspace
exercise_specialist = {
    "name": "exercise-specialist",
    "description": "Expert in exercise science, workout programming, and physical fitness. Use for exercise-related questions and plan creation.",
    "system_prompt": """You are an exercise specialist with expertise in:
- Workout programming for different fitness levels
- Exercise form and safety
- Progressive overload principles
- Recovery and injury prevention

Always consider the user's fitness level and any physical limitations.
Provide clear, actionable exercise instructions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

nutrition_specialist = {
    "name": "nutrition-specialist",
    "description": "Expert in nutrition science, meal planning, and dietary optimization. Use for food-related questions and meal plans.",
    "system_prompt": """You are a nutrition specialist with expertise in:
- Macro and micronutrient balance
- Meal planning and preparation
- Dietary restrictions and alternatives
- Nutrition timing for performance

Always respect dietary restrictions and preferences.
Focus on practical, achievable meal suggestions.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

mindfulness_specialist = {
    "name": "mindfulness-specialist",
    "description": "Expert in stress management, sleep optimization, and mental wellness. Use for stress, sleep, and mental health questions.",
    "system_prompt": """You are a mindfulness and mental wellness specialist with expertise in:
- Stress reduction techniques
- Sleep hygiene and optimization
- Meditation and breathing exercises
- Work-life balance strategies

Be supportive and non-judgmental.
Provide practical techniques that can be implemented immediately.""",
    "tools": [],  # Uses built-in file tools from backend
    "model": "openai:gpt-4o-mini",
}

print("Specialist subagents defined!")

Specialist subagents defined!


In [33]:
# Create the Wellness Coach coordinator
wellness_coach = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        # Planning
        write_todos,
        update_todo,
        list_todos,
        # Long-term Memory
        get_user_profile,
        save_user_preference,
        # Skills
        load_skill,
    ],
    backend=filesystem_backend,  # All file ops go to workspace
    subagents=[exercise_specialist, nutrition_specialist, mindfulness_specialist],
    system_prompt="""You are a Personal Wellness Coach that coordinates comprehensive wellness programs.

## Your Role
- Understand each user's unique goals, constraints, and preferences
- Create personalized, multi-week wellness programs
- Coordinate between exercise, nutrition, and mindfulness specialists
- Track progress and adapt recommendations

## Workflow
1. **Initial Assessment**: Get user profile and understand their situation
2. **Planning**: Create a todo list for the program components
3. **Delegation**: Use specialists for domain-specific content:
   - exercise-specialist: Workout plans and fitness guidance
   - nutrition-specialist: Meal plans and dietary advice
   - mindfulness-specialist: Stress and sleep optimization
4. **Integration**: Combine specialist outputs into a cohesive program
5. **Documentation**: Save all plans and recommendations to files

## Important
- Always check user profile first for context
- Respect any medical conditions or dietary restrictions
- Provide clear, actionable recommendations
- Save progress to files so users can reference later"""
)

print("Wellness Coach created with all 4 Deep Agent elements!")

Wellness Coach created with all 4 Deep Agent elements!


In [34]:
# Test the complete system
TODO_STORE.clear()

result = wellness_coach.invoke({
    "messages": [{
        "role": "user",
        "content": """Hi! My user_id is user_alex. I'd like you to create a 2-week wellness program for me.

I want to focus on:
1. Building a consistent exercise routine (I can exercise 3x per week for 30 mins)
2. Improving my diet (remember I'm vegetarian)
3. Better managing my work stress and improving my sleep

Please create comprehensive plans for each area and save them as separate files I can reference."""
    }]
})

print("Wellness Coach response:")
print(result["messages"][-1].content)

Wellness Coach response:
Perfect! Your comprehensive 2-week wellness program is complete! Here's what I've created for you:

## 🎯 **Your Complete Wellness Program Files**

**1. Master Program** - `/alex_2week_master_wellness_program.txt`
- Your complete integrated guide with daily schedules, examples, and success tracking
- Quick reference guides for stress management and energy optimization  
- Emergency protocols for challenging days
- Long-term maintenance strategies

**2. Exercise Program** - `/2_week_exercise_program_for_alex.txt`
- 6 detailed workout sessions (Mon/Wed/Fri for 2 weeks)
- Morning-friendly 30-minute routines
- Progression from Week 1 to Week 2
- Warm-ups, exercises, and cool-downs with clear instructions

**3. Vegetarian Meal Plan** - `/comprehensive_vegetarian_meal_plan.txt`
- 14 days of complete meals (breakfast, lunch, dinner, 2 snacks)
- Energy-boosting and sleep-promoting vegetarian foods
- Anxiety-friendly options with magnesium-rich foods
- Shopping lists and

In [35]:
# Review what was created
print("=" * 60)
print("FINAL TODO STATUS")
print("=" * 60)
print(list_todos.invoke({}))

print("\n" + "=" * 60)
print("GENERATED FILES")
print("=" * 60)
for f in sorted(WORKSPACE.iterdir()):
    if f.is_file():
        print(f"  [FILE] {f.name} ({f.stat().st_size} bytes)")
    elif f.is_dir():
        print(f"  [DIR] {f.name}/")

FINAL TODO STATUS
✅ [todo_1] Create comprehensive 2-week exercise program (completed)
✅ [todo_3] Develop vegetarian meal plan (completed)
✅ [todo_5] Design stress management and sleep optimization plan (completed)
✅ [todo_7] Create master wellness program document (completed)
✅ [todo_9] Save individual specialist plans as reference files (completed)

GENERATED FILES
  [FILE] 2_week_exercise_program_for_alex.txt (4884 bytes)
  [FILE] Alex_2_Week_Exercise_Program.txt (4541 bytes)
  [FILE] alex_2week_master_wellness_program.txt (7059 bytes)
  [FILE] alex_exercise_program.txt (4046 bytes)
  [FILE] alex_integrated_2week_wellness_program.txt (11071 bytes)
  [FILE] alex_nutrition_plan.txt (6181 bytes)
  [FILE] alex_nutrition_plan_week_1.txt (5451 bytes)
  [FILE] alex_nutrition_plan_week_2.txt (3222 bytes)
  [FILE] alex_stress_management_sleep_program.txt (8373 bytes)
  [FILE] complete_morning_routine_energy_guide_2024.md (15151 bytes)
  [FILE] comprehensive_morning_routine_guide.md (15151 byt

In [36]:
# Read one of the generated files
files = list(WORKSPACE.glob("*.md"))
if files:
    print(f"\nContents of {files[0].name}:")
    print("=" * 60)
    print(files[0].read_text()[:2000] + "..." if len(files[0].read_text()) > 2000 else files[0].read_text())


Contents of complete_morning_routine_energy_guide_2024.md:
# The Complete Morning Routine Guide for Better Energy

*Transform your mornings, transform your life*

## Introduction

Your morning routine sets the tone for your entire day. Research shows that the first few hours after waking are crucial for regulating your energy levels, mood, and cognitive performance. A well-designed morning routine isn't just about productivity—it's about creating a foundation for sustained energy and well-being.

Studies consistently demonstrate that people with structured morning routines report higher energy levels, better mood regulation, and improved overall life satisfaction. This guide combines the latest scientific research with practical, actionable steps to help you create a morning routine that energizes your body and mind.

## The Science Behind Morning Energy

### 🧠 **Circadian Rhythm Reset**
Your internal body clock responds to light and activities in the morning. Early light exposure and

---
## ❓ Question #3:

What are the key considerations when designing **subagent configurations**?

Consider:
- When should subagents share tools vs have distinct tools?
- How do you decide which model to use for each subagent?
- What's the right granularity for subagent specialization?

##### Answer:
While designing a subagent architecture the goal is to reduce cognitive load on any single agent, which minimizes hallucinations and improves task execution.

1. Share tools vs have distinct tools:
This depends on the scope of authority and the risk of confusion.
    a. Distinct Tools (The Specialist Model): Give subagents unique tools when tasks require deep focus. This prevents "tool bloat," where an agent has 20+ tools and struggles to pick the right one.
    b. Shared Tools (The Redundancy Model): Share tools when multiple agents perform similar operations but in different contexts.
    c. The Golden Rule: If a tool requires a specific "persona" or strict safety guardrails (like a delete_database tool), keep it isolated to a single, highly-constrained subagent.

2. Model Selection Stratergy:
    a. For Orchestration or decision router roles, a high-reasoning Large model is suggested. Because the role requires to understand complex intent and delegate without errors. 
    b. For data extractor/parser roles, fast/efficient small model is suggested. As it can help with tasks with clear patterns (JSON extraction) don't need massive parameters.
    c. For creative writer role a High-Context/Nuanced model should be good. Requires a model with a better "voice" and longer context window.
    d. For Code Executor role a coding specific model would be good. Models fine-tuned on Python/Java provide better syntax accuracy.
    
3. Granularity of Specialization
Finding the "Goldilocks zone" for subagent size is the hardest part of the design.
    a. Too Broad (The Monolith): One agent handles research, drafting, and formatting.
        Risk: The agent loses track of instructions midway through (instruction drift).
    b. Too Granular (The Micro-Manager): An agent just to "check for typos," another just to "check for commas."
        Risk: Massive "handoff overhead." Every time agents talk, you consume tokens and risk losing context in the transfer.
    c. The Sweet Spot: Define agents by output milestones. If a task results in a definitive "artifact" (a research brief, a code block, a formatted email), that's usually a good boundary for a subagent

4. State Management and Memory
When agents hand off tasks, they can lose the "why" behind a request.
    a. Short-term: Use a shared "Thread" or "State" object where agents can read previous notes.
    b. Long-term: Use a "Summary Agent" to condense long conversations before passing them to the next subagent to prevent hitting context limits.

## ❓ Question #4:

For a **production wellness application** using Deep Agents, what would you need to add?

Consider:
- Safety guardrails for health advice
- Persistent storage (not in-memory)
- Multi-user support and isolation
- Monitoring and observability
- Cost management with subagents

##### Answer:
1. Safety Guardrails & Clinical Alignment
    In wellness, "hallucinations" aren't just bugs; they are liabilities. You need a multi-layered safety sandwich.
    a. Preamble & Constitutional AI: Hard-code "Never provide medical diagnoses" into the system prompt.
    b. The "Jailbreak" Layer: Use a dedicated guardrail model (like Llama Guard) to intercept prompts that try to bypass safety filters.
    c. Fact-Checking RAG: Ground the agent in a "Gold Standard" knowledge base (e.g., PubMed, Peer-reviewed journals). If the agent’s reasoning deviates from these documents, the response is flagged.
    d. Emergency Triggering: Hard-coded keyword detection for self-harm or acute medical crises that bypasses the agent entirely and provides immediate hotlines/resources.
2. Persistent Storage 
    For a wellness app, memory isn't just about what was said; it’s about longitudinal progress.
    a. Vector Database (Pinecone/Weaviate): For "Semantic Memory"—retrieving relevant health context from weeks ago.
    b. Relational Database (PostgreSQL): For "Structured Memory"—storing user profiles, biometrics (sleep scores, heart rate), and subscription status.
    c. State Management: Use a framework like LangGraph or Temporal to persist the agent's state. If the server crashes mid-thought, the agent should resume exactly where it left off.
3. Multi-User Support & Isolation
    You cannot risk "cross-talk" where User A’s health data leaks into User B’s context.
    a. Tenant Isolation: Use Namespacing within your vector database. Every query must be hard-scoped to a user_id.
    b. Stateless Execution: The agent itself should be stateless. All context must be injected from the secure database at the start of the "turn" and cleared immediately after.
    c. Encryption at Rest: Ensure all health-related logs are encrypted (AES-256) to meet HIPAA or GDPR standards.

---
## 🏗️ Activity #2: Build a Wellness Coach Agent

Build your own wellness coach that uses all 4 Deep Agent elements.

### Requirements:
1. **Planning**: Create todos for a 30-day wellness challenge
2. **Context Management**: Store daily check-in notes
3. **Subagents**: At least 2 specialized subagents
4. **Memory**: Remember user preferences across interactions

### Challenge:
Create a "30-Day Wellness Challenge" system that:
- Generates a personalized 30-day plan
- Tracks daily progress
- Adapts recommendations based on feedback
- Saves a weekly summary report

In [42]:
### YOUR CODE HERE ###

# Step 1: Define your subagent configurations

# Subagent 1: Writes a personalized 30-day wellness plan
plan_writer_subagent = {
    "name": "30day-plan-writer",
    "description": "Use this agent to create a personalized 30-day wellness challenge plan. It writes the full plan to a file based on user goals, preferences, and constraints.",
    "system_prompt": """You are a 30-day wellness plan specialist. Your job is to:
- Create a detailed, day-by-day or week-by-week 30-day wellness challenge
- Use the user's goals, preferences, dietary restrictions, and fitness level from the request
- Include specific daily actions (exercise, nutrition, mindfulness, sleep) where appropriate
- Write the complete plan to a file in the workspace (e.g. 30day_wellness_plan_<user>.txt or .md)
- Structure the plan so it can be tracked (e.g. Week 1–4 or Day 1–30)
- Keep recommendations realistic and progressive (build difficulty over time)

Return a short summary of what you created and where you saved it.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

# Subagent 2: Tracks progress, saves check-ins, adapts from feedback, writes weekly summaries
progress_tracker_subagent = {
    "name": "progress-tracker",
    "description": "Use this agent to record daily check-ins, adapt the wellness plan based on user feedback, and write weekly summary reports. It reads and writes files in the workspace.",
    "system_prompt": """You are a wellness progress and adaptation specialist. Your job is to:
- Save daily check-in notes to a file (e.g. checkins/day_05.md or user_checkins.md) with date and user's notes
- Read existing check-ins and the 30-day plan when asked to adapt recommendations
- Suggest concrete adjustments to the plan based on feedback (e.g. "too intense", "skip mornings", "focus on sleep")
- Write a weekly summary report to a file (e.g. weekly_summary_week1.md) covering: what was done, wins, challenges, and next week focus
- Be supportive and specific; reference the user's own words from check-ins when adapting

Always write check-ins and summaries to the workspace so they persist. Return a short confirmation of what you saved.""",
    "tools": [],
    "model": "openai:gpt-4o-mini",
}

# Step 2: Create any additional tools you need
# Optional: custom tools for check-ins/summaries; the file backend already gives read/write.

# Step 3: Build the main coordinator agent
wellness_coach_30day = create_deep_agent(
    model=init_chat_model("anthropic:claude-sonnet-4-20250514"),
    tools=[
        write_todos,
        update_todo,
        list_todos,
        get_user_profile,
        save_user_preference,
    ],
    backend=filesystem_backend,
    subagents=[plan_writer_subagent, progress_tracker_subagent],
    system_prompt="""You are a 30-Day Wellness Challenge Coach. You use planning, context (files), subagents, and memory.

## Workflow
1. **Start**: Get user profile (get_user_profile) and create a todo list for the 30-day challenge (write_todos).
2. **Create plan**: Delegate to 30day-plan-writer to generate and save a personalized 30-day plan. Mark the todo done when complete.
3. **Check-ins**: When the user gives a daily check-in, delegate to progress-tracker to save the check-in and optionally adapt the plan. Use save_user_preference for any new preferences they mention.
4. **Weekly summary**: At the end of each week, delegate to progress-tracker to write a weekly summary report to the workspace.
5. **Adapt**: If the user says something is too hard, not working, or wants changes, delegate to progress-tracker with the feedback and the plan path so it can suggest and save adaptations.

## Subagents
- 30day-plan-writer: Creating the initial 30-day plan and saving it to a file.
- progress-tracker: Saving daily check-ins, adapting the plan from feedback, writing weekly summary reports.

Always check user profile first. Remember preferences with save_user_preference. Save all outputs to the workspace."""
)

print("Wellness Coach (30-day) created with plan_writer + progress_tracker subagents.")

# Step 4: Test with a user creating their 30-day challenge
TODO_STORE.clear()
result = wellness_coach_30day.invoke({
    "messages": [{
        "role": "user",
        "content": """My user_id is user_alex. Create my personalized 30-day wellness challenge.
I want to focus on: consistent morning movement (3x/week), better sleep, and less stress.
I'm vegetarian and prefer 20–30 min sessions. Please create the plan and save it to the workspace in plans directory."""
    }]
})
print("Step 4 response:", result["messages"][-1].content[:500] + "..." if len(result["messages"][-1].content) > 500 else result["messages"][-1].content)

# Step 5: Simulate a daily check-in and adaptation
# Uncomment and run after Step 4 (use the same thread or a new one with user_alex):
result2 = wellness_coach_30day.invoke({
     "messages": [
         {"role": "user", "content": "My user_id is user_alex. It's day 5. Check-in: I did 2 of 3 workouts, sleep was better, but mornings are still hard. Save this and adapt my plan if needed. Save to separate file so updated plan can be compared with original plan. "}
     ]
 })
print("Step 5 response:", result2["messages"][-1].content)


Wellness Coach (30-day) created with plan_writer + progress_tracker subagents.
Step 4 response: Perfect! Your personalized 30-day wellness challenge has been created and saved successfully! 🎉

## Your Challenge Overview:
- **Focus Areas**: Morning movement (3x/week), better sleep, and stress reduction
- **Session Length**: 20-30 minutes each
- **Dietary Approach**: Vegetarian-friendly throughout
- **Special Considerations**: Activities designed to help with mild anxiety

The plan is structured into 4 weekly themes:
1. **Week 1**: Building foundational routines
2. **Week 2**: Expanding move...
Step 5 response: Perfect! Here's what I've accomplished for your day 5 check-in:

## ✅ Check-in Saved
Your progress has been recorded showing:
- **Workouts**: 2 out of 3 completed (great progress!)
- **Sleep**: Improvement noted (fantastic news!)
- **Challenge**: Mornings still difficult

## 🔄 Plan Adapted
I've created an adapted version of your wellness plan (`/alex_30day_wellness_plan_adapted.md

---
## Summary

In this session, we explored **Deep Agents** and their four key elements:

| Element | Purpose | Implementation |
|---------|---------|----------------|
| **Planning** | Track complex tasks | `write_todos`, `update_todo`, `list_todos` |
| **Context Management** | Handle large contexts | File system tools, automatic offloading |
| **Subagent Spawning** | Delegate to specialists | `task` tool with custom configs |
| **Long-term Memory** | Remember across sessions | LangGraph Store integration |

### Key Takeaways:

1. **Deep Agents handle complexity** - Unlike simple tool loops, they can manage long-horizon, multi-step tasks
2. **Planning is context engineering** - Todo lists and files aren't just organization—they're extended memory
3. **Subagents prevent context bloat** - Delegation keeps the main agent focused and efficient
4. **Skills enable progressive disclosure** - Load capabilities on-demand instead of upfront
5. **The CLI makes interaction natural** - Interactive sessions with conversation resume

### Deep Agents vs Traditional Agents

| Aspect | Traditional Agent | Deep Agent |
|--------|-------------------|------------|
| Task complexity | Simple, single-step | Complex, multi-step |
| Context management | All in conversation | Files + summaries |
| Delegation | None | Subagent spawning |
| Memory | Within thread | Across sessions |
| Planning | Implicit | Explicit (todos) |

### When to Use Deep Agents

**Use Deep Agents when:**
- Tasks require multiple steps or phases
- Context would overflow in a simple loop
- Specialization would improve quality
- Users need to resume sessions
- Long-term memory is valuable

**Use Simple Agents when:**
- Tasks are straightforward Q&A
- Single tool call suffices
- Context fits easily
- No need for persistence

### Further Reading

- [Deep Agents Documentation](https://docs.langchain.com/oss/python/deepagents/overview)
- [Deep Agents GitHub](https://github.com/langchain-ai/deepagents)
- [Context Management Blog Post](https://www.blog.langchain.com/context-management-for-deepagents/)
- [Building Multi-Agent Applications](https://www.blog.langchain.com/building-multi-agent-applications-with-deep-agents/)
- [LangGraph Memory Concepts](https://langchain-ai.github.io/langgraph/concepts/memory/)